In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
lyrics = pd.read_csv("/kaggle/input/bad-bunny-lyrics/bad_bunny_lyrics.csv")

In [3]:
# Good regex resource: https://realpython.com/regex-python/#python-regex-metacharacters
import re

In [4]:
# practice_str = '[Letra de "NI BIEN NI MAL"]  [Intro: Bad Bunny] Yeh-yeh Yeh-yeh Yeh-yeh'
practice_song_lyric = lyrics.iloc[0,4]
practice_song_lyric

In [5]:
print(practice_song_lyric)

In [6]:
# Source: https://stackoverflow.com/questions/65022050/cleaning-song-lyrics-with-regex

In [7]:
# removed brackets used to denote sections of song
# removed new lines and added periods for sentence tokenization
removed_brackets = re.sub(r"[\[].*?[\]]","",practice_song_lyric)
no_brackets_or_new_lines = re.sub(r"\n{2}","",removed_brackets)
no_brackets_or_new_lines = re.sub(r"[\n]",". ",no_brackets_or_new_lines)
print(no_brackets_or_new_lines)

In [8]:
# might need to replace new lines with periods
correct_vas = re.sub(r"va\'","vas",no_brackets_or_new_lines)
correct_para_zero = re.sub(r"Pa\'l","Para el",correct_vas)
correct_para = re.sub(r"Pa\'","Para ",correct_para_zero)
correct_voy = re.sub(r"vo\'","voy ",correct_para)
correct_para_2 = re.sub(r"pa\'","para",correct_voy)
correct_de = re.sub(r"\'e","de",correct_para_2)
correct_puyas = re.sub(r"puya\'","puyas",correct_de)
correct_puyas += '.'
correct_puyas

In [9]:
!pip install stanza 

In [10]:
# Source: https://stanfordnlp.github.io/stanza/
# Source: https://stanfordnlp.github.io/stanza/tokenize.html#:~:text=Training%2DOnly%20Options-,Description,invoked%20by%20the%20name%20tokenize%20.
# Note: Stanza does not offer a Spanish Sentiment Analyzer
# Check out Stanza's other sections like:
# - Pipeline and Processors
# - Part-of-Speech and Morphological Features
# - Lemmatization
import stanza

In [11]:
stanza.download('es', package='ancora', processors='tokenize,mwt,pos,lemma', verbose=True)

In [12]:
stNLP = stanza.Pipeline(processors='tokenize,mwt,pos,lemma', lang='es', use_gpu=True)

In [13]:
doc = stNLP('Barack Obama nació en Hawaii.')
print(*[f'word: {word.text+" "}\tlemma: {word.lemma}' for sent in doc.sentences for word in sent.words], sep='\n')

In [14]:
# Source: https://stanfordnlp.github.io/stanza/tokenize.html#:~:text=Training%2DOnly%20Options-,Description,invoked%20by%20the%20name%20tokenize%20.
# tokenization and sentence segmentation performed
doc = stNLP(correct_puyas)

In [ ]:
print(*[f'word: {word.text+" "}\tlemma: {word.lemma}' for sent in doc.sentences[4:6] for word in sent.words], sep='\n')

In [ ]:
for i, sentence in enumerate(doc.sentences[4:6]):
    print(f'====== Sentence {i+1} tokens =======')
    print(*[f'id: {token.id}\ttext: {token.text}' for token in sentence.tokens], sep='\n')

In [ ]:
print([sentence.text for sentence in doc.sentences[4:6]])

In [ ]:
doc.sentences[4:5]

In [ ]:
# replaces all Unicode whitespace (such as \u) with a single space
#re.sub("\s+", " ", lyric, flags=re.UNICODE)

In [ ]:
# Spanish word normalization Python packages and libraries (Cucco)
# ftfy - handles mojibake

0. Sentence Tokenization - nltk package (tokenize and FreqDist)
0. Finding semantically related words using word embeddings (already loadded the dataset)
    * Look at Google Colab book on word2vec to see an example of how to do this
0. Spanish lemmatizer - https://stackoverflow.com/questions/60534999/how-to-solve-spanish-lemmatization-problems-with-spacy
    * The link above has sources that show how to make your own lemmatizer, FreeLing (provides lemmatization in Spanish and other languages), and spacy-stanza ("spaCy's API with the Stanza's models")
    * https://github.com/pablodms/spacy-spanish-lemmatizer    
    * pattern is another library
0. Word Cloud
1. **Topic Modeling** - can later let the user generate songs based on a topic
    * This might use zero-shot classification (which can be done in Hugging Face)
2. **Sentiment Analysis** - user can create songs with a specific sentiment
    * Another option for sentiment analysis: using pysentimiento for emotion analysis (in addition to sentiment analysis)
    * Another option for sentiment analysis: using Big Science Bloom
3. **Text Generation**
    * Two GPT-2 Models
3. Learn how to Use Kaggle Console effectively for projects
4. **Quick Tour on HuggingFace Transformers**
    * https://huggingface.co/docs/transformers/quicktour
5. **Getting Started with HuggingFace**
    * https://www.youtube.com/watch?v=QEaBAZQCtwE&t=120s

* How to track Jupyter notebook changes in GitHub - https://blog.reviewnb.com/jupyter-notebook-on-github-oss-examples/
* Top 8 Python libraries for NLP - https://www.analyticsvidhya.com/blog/2021/05/top-8-python-libraries-for-natural-language-processing-nlp-in-2021/

# Sentiment Analysis (Library #1)

In [ ]:
!pip install sentiment-analysis-spanish
!pip install keras tensorflow

In [ ]:
from sentiment_analysis_spanish import sentiment_analysis

In [ ]:
sentiment = sentiment_analysis.SentimentAnalysisSpanish()

In [ ]:
# Numbers close to zero are negative
# Numbers close to 1 are positive
# Numbers close to 0.5 are neutral
# Results made me think about: "It's not what you say but how you say it"
print(*[f'The sentence:\t {sentence.text} has a sentiment of:\t{sentiment.sentiment(sentence.text)}' for sentence in doc.sentences[20:30]], 
      sep='\n')

# Sentiment Analysis (Library #2)

In [ ]:
!pip install pysentimiento

In [ ]:
from pysentimiento import create_analyzer
analyzer = create_analyzer(task="sentiment", lang="es")

In [ ]:
print(*[f'The sentence:\t {sentence.text} has a sentiment of:\t{analyzer.predict(sentence.text)}' for sentence in doc.sentences[20:22]], 
      sep='\n')

In [ ]:
for sentence in doc.sentences[20:30]:
    analyzer_output_obj = analyzer.predict(sentence.text)
    print("The sentence is: ",sentence.text,"\n", analyzer_output_obj.probas)

In [ ]:
emotion_analyzer = create_analyzer(task="emotion", lang="es")

In [ ]:
# can print the most likely emotion 
for sentence in doc.sentences[20:30]:
    analyzer_output_obj = emotion_analyzer.predict(sentence.text)
    print("The sentence is: ", sentence.text,"\n","The emotion is: ", analyzer_output_obj.output)

In [ ]:
# can also output the probability of emotions
for sentence in doc.sentences[20:30]:
    analyzer_output_obj = emotion_analyzer.predict(sentence.text)
    print("The sentence is: ", sentence.text,"\n","The emotion probabilities are" analyzer_output_obj.probas)

# HuggingFace (BigScience Bloom)

In [ ]:
# Source: https://huggingface.co/docs/transformers/quicktour

In [15]:
!pip install torch

In [16]:
!pip install tensorflow

In [17]:
import torch
from transformers import pipeline

In [ ]:
from transformers import AutoTokenizer, AutoModel, 
model_name = "bigscience/bloom"
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
# PyTorch
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [ ]:
# TensorFlow
from transformers import TFAutoModelForSequenceClassification
model = TFAutoModelForSequenceClassification.from_pretrained(model_name)

In [ ]:
sentiment_classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

In [ ]:
sentiment_classifier("Yo te quiero, Puerto Rico.")

In [ ]:
# continue reading HuggingFace documentation - AutoClass section

# Two Approaches to Text Generation

## HuggingFace (DeepEsp GPT-2 Spanish)

In [18]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Source: Huggingface.co The most important thing to remember though is you need to instantiate the tokenizer with the 
# same model name to ensure you’re using the same tokenization rules a model was pretrained with.
# So, use the pretrained model's associated pretrained tokenizer - This ensures the text is split the 
# same way as the pretraining corpus, and uses the same corresponding tokens-to-index 
# (usually referrred to as the vocab) during pretraining.

# the AutoTokenizer method also calls the vocab
tokenizer = AutoTokenizer.from_pretrained("DeepESP/gpt2-spanish")
model = AutoModelForCausalLM.from_pretrained("DeepESP/gpt2-spanish")

## HuggingFace (Datificate GPT-2 Small Spanish)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("datificate/gpt2-small-spanish")

model = AutoModelForCausalLM.from_pretrained("datificate/gpt2-small-spanish")

## Quick Tour

In [19]:
from transformers import pipeline

In [24]:
print([sentence.text for sentence in doc.sentences[4:7]])

In [25]:
example_list_of_sentences = [sentence.text for sentence in doc.sentences[4:7]]

In [32]:
# sentences that are not the longest sentence are padded with zeroes (see 'attention_mask' key)
# the tensors are fed into the model
encoding = tokenizer(example_list_of_sentences, padding=True, return_tensors='pt')
print(encoding)

# my_outputs = model(encoding)

In [23]:
# decoding the input_ids to return to the original input
tokenizer.decode(encoding['input_ids'])

In [22]:
# currently not used in making the text predictions below
generator = pipeline("text-generation", model='DeepESP/gpt2-spanish', tokenizer=tokenizer)

In [ ]:
generator("Pase lo que pase no te voy")

In [ ]:
# Source: https://huggingface.co/docs/transformers/main_classes/text_generation
# greedy decoding
prompt = "Pase lo que pase no te voy"
input_ids = tokenizer(prompt, return_tensors='pt').input_ids

outputs = model.generate(input_ids, do_sample=False, max_length=60)
tokenizer.batch_decode(outputs, skip_special_tokens=True)

Once your model is fine-tuned, you can save it with its tokenizer.

Resource: Continue Hugging Face Tutorials from [here](https://huggingface.co/docs/transformers/training).

Spanish text-to-speech [model](https://huggingface.co/facebook/tts_transformer-es-css10) 